In [ ]:
# 참고
# https://porimp.tistory.com/entry/%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D-2020%EB%85%84%EB%8F%84-%EB%B2%84%EA%B1%B0%EC%A7%80%EC%88%98-%EB%A7%8C%EB%93%A4%EA%B8%B03
# https://hoho325.tistory.com/265
# https://nostalgiaa.tistory.com/36
# https://velog.io/@sukqbe/API-%EA%B5%AC%EA%B8%80-%EC%A7%80%EB%8F%84Google-Map-%EC%B6%94%EA%B0%80%ED%95%98%EA%B8%B0-API-Key-%EB%B0%9C%EA%B8%89%EB%B0%9B%EA%B8%B0-qumur49u


In [ ]:
#필요한 모듈 불러오기
import time #시간모듈
from itertools import count #수 관련 모듈


#웹크롤링 및 데이터 정리를 위한 준비
import urllib.request #웹에 접근하기 위한 모듈
import json #json 데이터 사용을 위한 모듈
from bs4 import BeautifulSoup as bs #웹크롤링을 위한 모듈
import pandas as pd #데이터 분석을 위한 모듈
import numpy as np #과학 계산을 위한 모듈
import scipy.stats #과학기술계산을 위한 모듈

!apt install chromium-chromedriver #웹드라이버 설치
!pip install selenium #셀레니움 설치
from selenium import webdriver #웹 테스트 자동화 도구, 웹 크롤링을 위한 모듈
#driver.implicitly_wait(3) 웹 로딩 기다려주는 시간

#url구문을 분석하기 위한 도구
import urllib.parse

#R사용을 위한 모듈
import rpy2.robjects as ro

#시각화를 위한 matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt

#마이너스 표시하기
mpl.rcParams['axes.unicode_minus'] = False 

#그래프 해상도 높이기
%config InlineBackend.figure_format = 'retina'

#matplotlib에 한글 폰트 적용
!apt -qq -y install fonts-nanum
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9) #한글폰트 기본으로 설정
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rc('font', size="40") #전체 폰트와 크기 설정
plt.rcParams['figure.figsize'] = (14,4) #차트 기본 크기 설정
mpl.font_manager._rebuild() #폰트설정 리빌드

In [ ]:
#HTTP요청 기능을 받아오는 모듈
req = urllib.request
#지역명 통일시켜주기 위한 딕셔너리
regionDic = {
    '서울':'서울특별시', '인천':'인천광역시', '대전':'대전광역시', '대구':'대구광역시',
    '부산':'부산광역시', '광주':'광주광역시', '울산':'울산광역시', '세종':'세종특별자치시'
    ,'경기':'경기도', '강원':'강원도', '충북':'충청북도', '충남':'충청남도', '경북':'경상북도',
    '경북':'경상북도', '경남':'경상남도', '전북':'전라북도', '전남':'전라남도', '제주':'제주특별자치도'
    }

In [ ]:
import sys
if 'google.colab' in sys.modules:
    !apt-get update
    !apt install chromium-chromedriver
    !cp /usr/lib/chromium-browser/chromedriver /usr/bin
    !pip install selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
#카카오맵에서 검색어 검색 후 크롤링을 위한 세팅

driver = webdriver.Chrome('chromedriver',options=options) #웹 드라이버를 크롬으로 실행
driver.implicitly_wait(3) #페이지 로딩을 위해 3초대기

driver.get('https://map.kakao.com/') #크롬드라이버로 카카오맵 접속
driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').clear()
#검색창 초기화
driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys('제기동 카페')
#검색창에 검색어 입력
driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/input').send_keys(Keys.ENTER)
#엔터를 통해 검색(click이 안됨)

# 장소명 '제기동 카페' (으)로 재검색 클릭 ---- 추가
driver.find_element_by_xpath('//*[@id="info.searchHeader.message"]/div/div[2]/a').send_keys(Keys.ENTER)

In [ ]:
driver.execute_script("document.body.style.zoom='80%'")#자바스크립트를 이용해 웹 사이즈 80%로 줄이기
time.sleep(5) # 5초 대기
driver.execute_script("window.scrollTo(0, 100)") #스크롤을 해서 제일 아래로 내리기
element = driver.find_element_by_id('info.search.place.more') #장소 더보기 찾음
driver.execute_script("arguments[0].click();", element) #장소 더보기 클릭
time.sleep(3) #3초 대기

In [ ]:
#카카오맵 페이지 한개 검색 후 데이터 정리
def kMap_one_page():
    html = driver.page_source
    soup = bs(html)
    RETURN_ARR = []

    for tag in soup.findAll('li', attrs={'class': 'PlaceItem'}):
        storeName = tag.findAll('a', attrs={'data-id': 'name'})[0].contents[-1]
        storeAddress = tag.findAll('p', attrs={'data-id': 'address'})[0].contents[-1]
        #storeAddress = storeAddress.split()[:2]
        storeAddress = storeAddress.split()
        RETURN_ARR.append([storeName] + storeAddress)

    for i in range(len(RETURN_ARR)):
        if regionDic.get(RETURN_ARR[i][1]) == None:
            pass
        else:
            RETURN_ARR[i][1] = regionDic.get(RETURN_ARR[i][1])

    #return pd.DataFrame(RETURN_ARR, columns=('store', 'addr1', 'addr2')) if RETURN_ARR else None
    return pd.DataFrame(RETURN_ARR, columns=('store', 'ad1','ad2','ad3','ad4','ad5','ad6','ad7')) if RETURN_ARR else None

In [ ]:
def kMap_all_page(lastPage):
    FINAL_RETURN = []
    for n in range(lastPage):
        
        pgNum = (n+1)%5
        print("{}페이지 진행중".format(n+1))
        
        if pgNum != 0:
            pgNum = pgNum 
        elif pgNum == 0:
            pgNum = 5
        
        driver.execute_script("window.scrollTo(0, 100)") #스크롤을 해서 제일 아래로 내리기
        nextPage = "info.search.page.no{}".format(pgNum)
        elem=driver.find_element_by_id(nextPage)
        driver.execute_script("arguments[0].click();", elem)
        time.sleep(3)
        FINAL_RETURN.append(kMap_one_page())
        if (pgNum % 5 == 0):
            driver.execute_script("window.scrollTo(0, 100)") #스크롤을 해서 제일 아래로 내리기
            elem=driver.find_element_by_id("info.search.page.next")
            driver.execute_script("arguments[0].click();", elem)
            time.sleep(3)
        else:
            pass
            
    print("크롤링이 완료됐습니다.")
    driver.quit()
    return pd.concat(FINAL_RETURN)

In [ ]:
bgk_table = kMap_all_page(2)
display(bgk_table['store'].duplicated())

In [ ]:
bgk_table

,store,ad1,ad2,ad3,ad4,ad5,ad6,ad7
0,카페 헤이피,서울특별시,동대문구,제기로2길,11,2층,None,None
1,카페파인,서울특별시,동대문구,안암로22길,15,형준빌딩,"1,",2층
2,프렌치카페소딜리셔스,서울특별시,동대문구,왕산로,109,2층,None,None
3,카페브레송,서울특별시,동대문구,안암로22길,6,None,None,None
4,카페BBC,서울특별시,동대문구,왕산로19라길,3,어반넥서스,1층,None
5,작심스터디카페 고려대정문점,서울특별시,동대문구,안암로26길,8,지하1층,None,None
6,프린트카페 고대정문점,서울특별시,동대문구,안암로,162-1,1층,None,None
7,카페 멜로우,서울특별시,동대문구,안암로24길,20-4,1층,None,None
8,카페비비씨,서울특별시,동대문구,왕산로19라길,3,None,None,None
9,다미가한방카페,서울특별시,동대문구,약령중앙로,5,대산빌딩,1층,None


In [ ]:
res = pd.DataFrame(columns=['name','addr'])
res['name'] = bgk_table['store']
res['addr'] = bgk_table['ad1'] + " " + bgk_table['ad2']+ " " + bgk_table['ad3'] + " " + bgk_table['ad4']
res = res.reset_index(drop=True)

In [ ]:
res

,name,addr
0,카페 헤이피,서울특별시 동대문구 제기로2길 11
1,카페파인,서울특별시 동대문구 안암로22길 15
2,프렌치카페소딜리셔스,서울특별시 동대문구 왕산로 109
3,카페브레송,서울특별시 동대문구 안암로22길 6
4,카페BBC,서울특별시 동대문구 왕산로19라길 3
5,작심스터디카페 고려대정문점,서울특별시 동대문구 안암로26길 8
6,프린트카페 고대정문점,서울특별시 동대문구 안암로 162-1
7,카페 멜로우,서울특별시 동대문구 안암로24길 20-4
8,카페비비씨,서울특별시 동대문구 왕산로19라길 3
9,다미가한방카페,서울특별시 동대문구 약령중앙로 5


In [ ]:
# 구글 키 필요 -> 27곳밖에 되지 않으므로 직접 좌표 입력함

!pip install googlemaps
import googlemaps

gmaps_key = "AlzaSyBb-qcA00eiqk_9JZZkcQyeQUhVcgG_5Xw"
gmaps = googlemaps.Client(key=gmaps_key)

place_lat = []
place_lng = []

max_lat = 38.0
min_lat = 33.0
max_lng = 132.0
min_lng = 126.0

for place in res['addr']:
  tmp = gmaps.geocode(place, language = "ko")

  # 검색 될 경우
  if tmp:
    tmp_loc = tmp[0].get("geometry")
    tmp_lat = tmp_loc["location"]["lat"]
    tmp_lng = tmp_loc["location"]["lng"]

    # 우리나라 밖일 경우
    if(tmp_lat > max_lat or tmp_lat < min_lat or tmp_lng > max_lng or tmp_lng < min_lng):
      place_lat.append(0)
      place_lng.append(0)

    else:
      place_lat.append(tmp_lat)
      place_lng.append(tmp_lng)

  else:
    place_lat.append(0)
    place_lng.append(0)

In [ ]:
# https://gooddata.go.kr//stdsvc/dataReg.do
# http://www.dawuljuso.com/

In [ ]:
xy = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/2021CCP/kt_data/data_folder/cafexy.xlsx')

In [ ]:
xy['name'] = res['name']
xy = xy[['name','경도','위도']]
xy.columns = ['name','x','y']

In [ ]:
xy

,name,x,y
0,카페 헤이피,127.033339,37.585924
1,카페파인,127.035669,37.588086
2,프렌치카페소딜리셔스,127.035937,37.578652
3,카페브레송,127.035081,37.588178
4,카페BBC,127.033686,37.578819
5,작심스터디카페 고려대정문점,127.036021,37.588929
6,프린트카페 고대정문점,127.035675,37.589019
7,카페 멜로우,127.035986,37.588093
8,카페비비씨,127.033686,37.578819
9,다미가한방카페,127.036584,37.578837


In [ ]:
xy.to_excel('/content/drive/MyDrive/Colab Notebooks/2021CCP/kt_data/data_folder/jegi_cafe.xlsx')